# 3.2 LangChain에서 도구(tool) 활용 방법

- LangChain에서 도구(tool)을 활용하는 방법을 알아봅니다
- 이 강의에서는 도구를 활용하는 방법을 중점적으로 다루며, 도구를 활용한 에이전트 개발 방법은 3.3 강의에서 다룹니다
    - LangGraph에서의 도구 활용 방법은 LangChain의 문법을 따르니 주의깊게 봐주세요

In [1]:
# from dotenv import load_dotenv

# load_dotenv(dotenv_path="../.env")

In [2]:
from langchain_ollama import ChatOllama

llm_llama31 = ChatOllama(model="llama3.1")
llm_qwen25 = ChatOllama(model="qwen2.5")

c:\Users\a1ljm\Desktop\ANSWER_25\inflearn-langgraph-agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- [tool decorator](https://python.langchain.com/docs/how_to/custom_tools/#tool-decorator)를 사용하면 쉽게 도구를 만들 수 있습니다

In [3]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a+b

@tool
def mul(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a*b

- LLM을 호출했을 때와 도구를 사용했을 때의 차이를 알아봅니다

In [4]:
query = "3 곱하기 5는?"

In [5]:
llm_llama31.invoke(query)

AIMessage(content='15입니다.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-12-20T23:08:18.032425Z', 'done': True, 'done_reason': 'stop', 'total_duration': 515659900, 'load_duration': 102998900, 'prompt_eval_count': 18, 'prompt_eval_duration': 136370200, 'eval_count': 4, 'eval_duration': 253410800, 'logprobs': None, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--019b3e05-5cc2-7e01-be9a-b66dcffa4120-0', usage_metadata={'input_tokens': 18, 'output_tokens': 4, 'total_tokens': 22})

In [6]:
llm_qwen25.invoke(query)

AIMessage(content='3 곱하기 5는 15입니다.', additional_kwargs={}, response_metadata={'model': 'qwen2.5', 'created_at': '2025-12-20T23:08:19.7825252Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1739342400, 'load_duration': 82197800, 'prompt_eval_count': 37, 'prompt_eval_duration': 681929600, 'eval_count': 13, 'eval_duration': 869145100, 'logprobs': None, 'model_name': 'qwen2.5', 'model_provider': 'ollama'}, id='lc_run--019b3e05-5ef9-75a0-874e-56b13d2e8d7f-0', usage_metadata={'input_tokens': 37, 'output_tokens': 13, 'total_tokens': 50})

- 도구 리스트는 LLM에 해당하는 `BaseModel` 클래스에 `bind_tools` 메서드를 통해 전달합니다

In [7]:
llm_with_tools = llm_qwen25.bind_tools([add, mul])
llm_with_tools

RunnableBinding(bound=ChatOllama(model='qwen2.5'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': '숫자 a와 b를 더합니다.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'mul', 'description': '숫자 a와 b를 곱합니다.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

- `AIMessage`의 `additional_kwargs` 속성은 `tool_calls`를 포함합니다
- `tool_calls`는 도구를 호출하는 메시지를 포함합니다

In [8]:
response = llm_with_tools.invoke(query)
response

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5', 'created_at': '2025-12-20T23:08:30.2612713Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10459572500, 'load_duration': 83806300, 'prompt_eval_count': 221, 'prompt_eval_duration': 7896727800, 'eval_count': 25, 'eval_duration': 2355797800, 'logprobs': None, 'model_name': 'qwen2.5', 'model_provider': 'ollama'}, id='lc_run--019b3e05-65d8-7571-bdd6-1afba1503955-0', tool_calls=[{'name': 'mul', 'args': {'a': 3, 'b': 5}, 'id': '7183264f-65b7-4fa3-be7d-35b77e6e7010', 'type': 'tool_call'}], usage_metadata={'input_tokens': 221, 'output_tokens': 25, 'total_tokens': 246})

- 여기서 `tool_calls`의 형태를 기억해두시면 남은 강의를 이해하시는데 도움이 됩니다

In [9]:
response.tool_calls

[{'name': 'mul',
  'args': {'a': 3, 'b': 5},
  'id': '7183264f-65b7-4fa3-be7d-35b77e6e7010',
  'type': 'tool_call'}]

In [10]:
from typing import Sequence
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage, ToolMessage

message_list: Sequence[AnyMessage] = []
# message_list: list[AnyMessage] = []  # 이렇게 해도 됨
human_message = HumanMessage(query)
message_list.append(human_message)
message_list

[HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={})]

- `tool_calls` 속성은 도구를 호출하는 메시지를 포함합니다
- `tool_calls`를 가진 `AIMessage`의 형태를 기억해두시면 남은 강의를 이해하시는데 도움이 됩니다

In [11]:
ai_message = llm_with_tools.invoke(message_list)
ai_message

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5', 'created_at': '2025-12-20T23:08:32.8991747Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2614423900, 'load_duration': 84456300, 'prompt_eval_count': 221, 'prompt_eval_duration': 66150400, 'eval_count': 25, 'eval_duration': 2372432200, 'logprobs': None, 'model_name': 'qwen2.5', 'model_provider': 'ollama'}, id='lc_run--019b3e05-8ecb-7d40-a9ed-4aceb920e291-0', tool_calls=[{'name': 'mul', 'args': {'a': 3, 'b': 5}, 'id': 'bab5a11a-484a-4022-af07-720a75ca97e3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 221, 'output_tokens': 25, 'total_tokens': 246})

In [12]:
ai_message.tool_calls

[{'name': 'mul',
  'args': {'a': 3, 'b': 5},
  'id': 'bab5a11a-484a-4022-af07-720a75ca97e3',
  'type': 'tool_call'}]

In [13]:
message_list.append(ai_message)
message_list

[HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5', 'created_at': '2025-12-20T23:08:32.8991747Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2614423900, 'load_duration': 84456300, 'prompt_eval_count': 221, 'prompt_eval_duration': 66150400, 'eval_count': 25, 'eval_duration': 2372432200, 'logprobs': None, 'model_name': 'qwen2.5', 'model_provider': 'ollama'}, id='lc_run--019b3e05-8ecb-7d40-a9ed-4aceb920e291-0', tool_calls=[{'name': 'mul', 'args': {'a': 3, 'b': 5}, 'id': 'bab5a11a-484a-4022-af07-720a75ca97e3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 221, 'output_tokens': 25, 'total_tokens': 246})]

- `AIMessage`의 `tool_calls`를 활용해서 도구를 직접 호출할 수도 있습니다

In [14]:
tool_message = mul.invoke(ai_message.tool_calls[0])
tool_message

ToolMessage(content='15', name='mul', tool_call_id='bab5a11a-484a-4022-af07-720a75ca97e3')

- 하지만 에이전트의 경우 도구를 직접 호출하는 것이 아니라 도구를 호출하는 메시지를 만들어서 전달합니다

In [15]:
message_list.append(tool_message)
llm_with_tools.invoke(message_list)

AIMessage(content='3 곱하기 5는 15입니다.', additional_kwargs={}, response_metadata={'model': 'qwen2.5', 'created_at': '2025-12-20T23:08:35.441222Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2509224600, 'load_duration': 79795500, 'prompt_eval_count': 263, 'prompt_eval_duration': 1161783400, 'eval_count': 13, 'eval_duration': 1239312000, 'logprobs': None, 'model_name': 'qwen2.5', 'model_provider': 'ollama'}, id='lc_run--019b3e05-9922-7e11-8a77-360990845b89-0', usage_metadata={'input_tokens': 263, 'output_tokens': 13, 'total_tokens': 276})

- `message_list`의 순서를 기억해두시면 남은 강의를 이해하시는데 도움이 됩니다

In [16]:
message_list

[HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5', 'created_at': '2025-12-20T23:08:32.8991747Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2614423900, 'load_duration': 84456300, 'prompt_eval_count': 221, 'prompt_eval_duration': 66150400, 'eval_count': 25, 'eval_duration': 2372432200, 'logprobs': None, 'model_name': 'qwen2.5', 'model_provider': 'ollama'}, id='lc_run--019b3e05-8ecb-7d40-a9ed-4aceb920e291-0', tool_calls=[{'name': 'mul', 'args': {'a': 3, 'b': 5}, 'id': 'bab5a11a-484a-4022-af07-720a75ca97e3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 221, 'output_tokens': 25, 'total_tokens': 246}),
 ToolMessage(content='15', name='mul', tool_call_id='bab5a11a-484a-4022-af07-720a75ca97e3')]